In [1]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sklearn
import math
from tensorflow.keras.models import Model

In [2]:
ds = tfds.load('deep_weeds', as_supervised= True, shuffle_files= True, batch_size = -1)

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [3]:
images, labels = ds['train']

In [4]:
del ds
#del images
#del labels

In [5]:
def preprocess(img):
    org = img
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=3)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=3)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    im_max = np.max(sobel)
    im_min = np.min(sobel)
    sobel = (sobel - im_min)/(im_max - im_min)
    sobel = sobel * 255
    sobel = cv2.GaussianBlur(sobel,(3,3),cv2.BORDER_DEFAULT)
    gray = np.expand_dims(gray, axis = 2)
    sobel = np.expand_dims(sobel, axis = 2)
    return np.concatenate([org,gray,sobel], axis = 2)

# Load Model

In [6]:
model  = tf.keras.models.load_model('CNN_Models/3_block_cnn/3_blockV2.h5')
model.compile()

In [7]:
model.summary()

Model: "2_block_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Original_Image (InputLayer)  [(None, 256, 256, 5)]     0         
_________________________________________________________________
random_flip_29 (RandomFlip)  (None, 256, 256, 5)       0         
_________________________________________________________________
random_rotation_29 (RandomRo (None, 256, 256, 5)       0         
_________________________________________________________________
rescaling_29 (Rescaling)     (None, 256, 256, 5)       0         
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 128, 128, 64)      2944      
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization_56 (Batc (None, 64, 64, 64)        

# Using Preprocessing

In [8]:
#Pick a random image
ind = np.random.randint(0, len(images))

#Format for Neural Network
test = images[ind].numpy()
pre = preprocess(test)
im = np.expand_dims(pre, axis = 0)

# Using Model to predict

In [9]:
prediction = model.predict(im)
print(f'Prediction is {np.argmax(prediction)}')
print(f'Label is {labels[ind]}')

Prediction is 3
Label is 3
